In [52]:
import pandas as pd

In [111]:
data_VIX = pd.read_csv('VIX.csv', parse_dates=['Date'])
data_Obama = pd.read_csv('BarackObama.csv', parse_dates=['Date'])

In [127]:
data_VIX['Date'].dtype

dtype('<M8[ns]')

In [113]:
data_Obama

,Date,author,content,number_of_likes,number_of_shares
0,2014-01-08 14:57:00,BarackObama,Make #53 special for #44: http://t.co/leGTnedv...,1512,726
1,2014-01-08 17:58:00,BarackObama,"Job growth has exceeded 200,000 for the sixth ...",816,556
2,2014-01-08 18:46:00,BarackObama,Happening now: President Obama delivers a stat...,491,288
3,2014-01-08 18:54:00,BarackObama,"""That's the longest streak of private sector j...",804,619
4,2014-01-08 18:57:00,BarackObama,"""The fact is we could be much further along .....",1505,1260
...,...,...,...,...,...
2858,2016-12-10 16:35:00,BarackObama,Let's leave our children and grandchildren a p...,12072,3692
2859,2016-12-10 17:57:00,BarackObama,"Last week, more than 190 countries reached a l...",3283,855
2860,2016-12-10 19:43:00,BarackObama,Support the bold steps we're taking against cl...,1979,496
2861,2016-12-10 21:02:00,BarackObama,LIVE: President Obama is speaking at a recepti...,1931,468


In [114]:
first_date = pd.to_datetime('1/1/2015')
last_date = pd.to_datetime('1/1/2017')

In [115]:
data_Obama = data_Obama[data_Obama['Date']<last_date]
data_Obama = data_Obama[data_Obama['Date']>first_date]
data_VIX = data_VIX[data_VIX['Date']<last_date]
data_VIX = data_VIX[data_VIX['Date']>first_date]

In [116]:
data_VIX

,Date,Price,Change
3773,2015-01-02,17.76,0.116279
3774,2015-01-05,19.19,0.080518
3775,2015-01-06,20.33,0.059406
3776,2015-01-07,20.15,-0.008854
3777,2015-01-08,17.93,-0.110174
...,...,...,...
4272,2016-12-23,11.38,0.005300
4273,2016-12-27,12.26,0.077329
4274,2016-12-28,11.89,-0.030179
4275,2016-12-29,13.15,0.105971


In [135]:
data_Obama['Date']= pd.to_datetime(data_Obama['Date'].dt.normalize())
data_VIX['Date']= pd.to_datetime(data_VIX['Date'].dt.normalize())

In [136]:
data_VIX['Date'].dtype

dtype('<M8[ns]')

In [118]:
df=data_Obama[['Date','number_of_likes']]

In [119]:
df

,Date,number_of_likes
643,2015-01-01,930
644,2015-01-01,1075
645,2015-01-01,1248
646,2015-01-01,11313
647,2015-01-04,1474
...,...,...
2857,2016-12-10,3211
2858,2016-12-10,12072
2859,2016-12-10,3283
2860,2016-12-10,1979


In [144]:
df = data_Obama.groupby('Date').sum()

In [145]:
df2 = pd.merge(df,data_VIX, how='inner', on='Date')

In [147]:
df2

,Date,number_of_likes,number_of_shares,Price,Change
0,2015-01-05,4182,3099,19.19,0.080518
1,2015-01-06,87467,57145,20.33,0.059406
2,2015-01-07,16168,6205,20.15,-0.008854
3,2015-01-08,9612,4116,17.93,-0.110174
4,2015-01-09,6820,3650,16.44,-0.083101
...,...,...,...,...,...
416,2016-12-02,16680,5269,14.16,0.056716
417,2016-12-05,21845,6662,13.75,-0.028955
418,2016-12-07,6614,1576,11.59,-0.049221
419,2016-12-08,40272,11954,12.10,0.044003


In [146]:
df2.corr()

,number_of_likes,number_of_shares,Price,Change
number_of_likes,1.000000,0.953408,-0.016306,0.031180
number_of_shares,0.953408,1.000000,-0.017743,0.039245
Price,-0.016306,-0.017743,1.000000,0.227343
Change,0.031180,0.039245,0.227343,1.000000
